In [118]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
import re
import random 
import numpy as np
import warnings
import tensorflow as tf 
from imblearn.over_sampling import RandomOverSampler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from preprocess_days_stats import preprocess_match_days
from preprocess_time_serie import preprocess_teams, preprocess_time_series_features
from tensorflow.keras import layers
from helper_functions_tensorflow import CSVLoggerCallback, CSVLoggerCallbackParams, plot_loss_curve

# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A_dump\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [119]:
df_giornate.head()

,div,giornata,stagione,date,hometeam,awayteam,ft_home_goals,ft_away_goals,ft_result,ht_home_goals,ht_away_goals,ht_results,home_shots,away_shots,home_shots_targ,away_shots_targ,home_fouls,away_fouls,home_corners,away_corners,home_yellow,away_yellow,home_red,away_red,home_win_odds,draw_odds,away_win_odds
0,I1,1,2005/2006,2005-08-27,Fiorentina,Sampdoria,2.0,1.0,H,2.0,0.0,H,15.0,9.0,9.0,4.0,18.0,23.0,9.0,3.0,1.0,4.0,0.0,0.0,2.20,2.87,3.25
1,I1,1,2005/2006,2005-08-27,Livorno,Lecce,2.0,1.0,H,1.0,1.0,D,17.0,6.0,7.0,5.0,27.0,21.0,9.0,2.0,2.0,3.0,0.0,0.0,1.90,2.90,4.00
2,I1,1,2005/2006,2005-08-28,Ascoli,Milan,1.0,1.0,D,0.0,0.0,D,8.0,16.0,3.0,9.0,22.0,16.0,0.0,5.0,2.0,1.0,0.0,0.0,7.50,3.75,1.40
3,I1,1,2005/2006,2005-08-28,Inter,Treviso,3.0,0.0,H,1.0,0.0,H,16.0,7.0,9.0,3.0,13.0,20.0,6.0,1.0,1.0,3.0,0.0,0.0,1.16,5.50,13.00
4,I1,1,2005/2006,2005-08-28,Juventus,Chievo,1.0,0.0,H,1.0,0.0,H,16.0,2.0,7.0,0.0,16.0,12.0,6.0,2.0,1.0,2.0,0.0,0.0,1.25,5.00,13.00


In [120]:
all_features = ['ft_goals','ft_goals_conceded','shots','shots_target', 'fouls_done','corners_obtained', 'yellows', 'reds']
less_features = ['ft_goals','ft_goals_conceded','shots', 'fouls_done','corners_obtained', 'reds']
few_features = ['ft_goals','ft_goals_conceded','shots', 'reds']

In [121]:
num_features = 'all'
Statistiche_squadre_dict = preprocess_teams(dataframe = df_giornate)
df_Serie_A = preprocess_time_series_features(num_features, Statistiche_squadre_dict, df_giornate, 5).dropna()
df_Serie_A[(df_Serie_A['hometeam']=='Lecce') | (df_Serie_A['awayteam']=='Lecce')].head(6)

NameError: name 'np' is not defined

In [ ]:
Train_df = df_Serie_A.iloc[:int(len(df_Serie_A)*0.85)]
Valid_df = df_Serie_A.iloc[int(len(df_Serie_A)*0.85):int(len(df_Serie_A)*0.97)]
Test_df = df_Serie_A.iloc[int(len(df_Serie_A)*0.95):]

len(Train_df),len(Test_df), (len(Valid_df))

(5104, 301, 720)

In [ ]:
Train_odds = Train_df[['home_win_odds','draw_odds','away_win_odds']]
Train_teams = Train_df[['hometeam','awayteam']]
Train_labels = Train_df[['ft_result']]

if num_features == 'all':
    features = all_features
    print('utilizzando tutte le features')
elif num_features == 'less':
    print('utilizzando meno features')
    features = less_features
else:
    print('utilizzando poche features')
    features=few_features

Train_dict_features={}

for feature in features:
    Train_dict_features[feature] = pd.DataFrame({})
    for colonna in Train_df.columns:
        pattern = re.compile(rf'^home_{feature}_\d+$')
        if pattern.match(colonna):
            Train_dict_features[feature][colonna]=Train_df[colonna]

Train_dict_features.keys()

utilizzando tutte le features


dict_keys(['ft_goals', 'ft_goals_conceded', 'shots', 'shots_target', 'fouls_done', 'corners_obtained', 'yellows', 'reds'])

In [ ]:
Valid_odds = Valid_df[['home_win_odds','draw_odds','away_win_odds']]
Valid_teams = Valid_df[['hometeam','awayteam']]
Valid_labels = Valid_df[['ft_result']]


if num_features == 'all':
    features = all_features
    print('utilizzando tutte le features')
elif num_features == 'less':
    print('utilizzando meno features')
    features = less_features
else:
    print('utilizzando poche features')
    features=few_features

Valid_dict_features={}

for feature in features:
    Valid_dict_features[feature] = pd.DataFrame({})
    for colonna in Valid_df.columns:
        pattern = re.compile(rf'^home_{feature}_\d+$')
        if pattern.match(colonna):
            Valid_dict_features[feature][colonna]=Valid_df[colonna]
    for colonna in Valid_df.columns:
        pattern = re.compile(rf'^away_{feature}_\d+$')
        if pattern.match(colonna):
            Valid_dict_features[feature][colonna]=Valid_df[colonna]
    
    

feature_list = []
for feature in list(Valid_dict_features.keys()):
    for campo in Valid_dict_features[feature].columns:
        feature_list.append(campo)
feature_list

utilizzando tutte le features


['home_ft_goals_1',
 'home_ft_goals_2',
 'home_ft_goals_3',
 'home_ft_goals_4',
 'home_ft_goals_5',
 'away_ft_goals_1',
 'away_ft_goals_2',
 'away_ft_goals_3',
 'away_ft_goals_4',
 'away_ft_goals_5',
 'home_ft_goals_conceded_1',
 'home_ft_goals_conceded_2',
 'home_ft_goals_conceded_3',
 'home_ft_goals_conceded_4',
 'home_ft_goals_conceded_5',
 'away_ft_goals_conceded_1',
 'away_ft_goals_conceded_2',
 'away_ft_goals_conceded_3',
 'away_ft_goals_conceded_4',
 'away_ft_goals_conceded_5',
 'home_shots_1',
 'home_shots_2',
 'home_shots_3',
 'home_shots_4',
 'home_shots_5',
 'away_shots_1',
 'away_shots_2',
 'away_shots_3',
 'away_shots_4',
 'away_shots_5',
 'home_shots_target_1',
 'home_shots_target_2',
 'home_shots_target_3',
 'home_shots_target_4',
 'home_shots_target_5',
 'away_shots_target_1',
 'away_shots_target_2',
 'away_shots_target_3',
 'away_shots_target_4',
 'away_shots_target_5',
 'home_fouls_done_1',
 'home_fouls_done_2',
 'home_fouls_done_3',
 'home_fouls_done_4',
 'home_fouls

In [ ]:
Test_odds = Test_df[['home_win_odds','draw_odds','away_win_odds']]
Test_teams = Test_df[['hometeam','awayteam']]
Test_labels = Test_df[['ft_result']]

if num_features == 'all':
    features = all_features
    print('utilizzando tutte le features')
elif num_features == 'less':
    print('utilizzando meno features')
    features = less_features
else:
    print('utilizzando poche features')
    features=few_features

Test_dict_features={}

for feature in features:
    Valid_dict_features[feature] = pd.DataFrame({})
    for colonna in Valid_df.columns:
        pattern = re.compile(rf'^home_{feature}_\d+$')
        if pattern.match(colonna):
            Valid_dict_features[feature][colonna]=Valid_df[colonna]
    for colonna in Valid_df.columns:
        pattern = re.compile(rf'^away_{feature}_\d+$')
        if pattern.match(colonna):
            Valid_dict_features[feature][colonna]=Valid_df[colonna]

Test_dict_features.keys()

In [ ]:
#Creo un column transformer, rende più facile normalizzare il dataframe
teams_transf = make_column_transformer(
(OneHotEncoder(handle_unknown='ignore'), ['hometeam','awayteam']),
sparse_threshold=0  
)

label_transf = make_column_transformer(
(OneHotEncoder(handle_unknown='ignore'), ['ft_result']),
sparse_threshold=0  
)

Train_dict_features_norm = Train_dict_features.copy()
feature_list = []
for feature in list(Train_dict_features.keys()):
    numerical_transf = make_column_transformer(
        (MinMaxScaler(), feature_list), 
        sparse_threshold=0  
            )
    numerical_transf.fit(Train_dict_features[feature])

        
numerical_transf = make_column_transformer(
(MinMaxScaler(), feature_list), 
sparse_threshold=0  
)

teams_transf.fit(Train_teams)
label_transf.fit(Train_dict_features)


NameError: name 'make_column_transformer' is not defined